In [3]:
#r "nuget: Azure.AI.OpenAI, 2.2.0-beta.4"
#r "nuget: Azure.Identity"
#r "nuget: Azure"
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Azure.Cosmos"
using Azure;
using Azure.AI.OpenAI;
using Azure.Identity;
using OpenAI.Embeddings;
using Newtonsoft.Json;
using Newtonsoft.Json.Linq;
using System.Net.Http;
using System.Collections.ObjectModel;
using Microsoft.Azure.Cosmos;

Installed Packages Azure, 1.0.6 Azure.AI.OpenAI, 2.2.0-beta.4 Azure.Identity, 1.14.0 Microsoft.Azure.Cosmos, 3.51.0 Newtonsoft.Json, 13.0.3

In [4]:
public class Post    
{
    public string id { get; set; }
    public int PostId { get; set; }
    public string PostBody { get; set; }
    public string Title { get; set; }
    public int ViewCount { get; set; }
    public int AnswerCount { get; set; }
    public int CommentCount { get; set; }
    public int FavoriteCount { get; set; }
    public int AcceptedAnswerId { get; set; }
    public DateTime? CreatedOn { get; set; }
    public DateTime? ClosedDate { get; set; }
    public int OwnerUserId { get; set; }
    public string OwnerDisplayName { get; set; }
    public string PostType { get; set; }
    public int Score { get; set; }
    public string Tags { get; set; }
    public float[] bodyvector {get;set;}
    public string score {get;set;}
}

In [5]:
var openAIClient = new AzureOpenAIClient(
    new Uri("https://savranweb.openai.azure.com/"),
    new AzureKeyCredential("3Qg3P8Fw37qaN3ZtG2JJapZZVjBFTKvOIDaIYXoL5a5fduu3wz2XJQQJ99BDACYeBjFXJ3w3AAABACOGPUlQ"));
var aiclient = openAIClient.GetEmbeddingClient("embedding");

In [6]:
var cstring = "AccountEndpoint=https://localhost:8081/;AccountKey=C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw==";
var client = new CosmosClient(cstring);
var db = client.GetDatabase("StackOverflow");
var vectorContainer = db.GetContainer("VectorPosts");

In [7]:
ReadOnlyMemory<float> GenerateVector(string text)
{    
    OpenAIEmbedding newembedding = aiclient.GenerateEmbedding(text);
    return newembedding.ToFloats();
}

In [8]:
float[] embedding = GenerateVector("Find the common database topics?").ToArray();
  var queryDef = new QueryDefinition(
      query: $"SELECT TOP 10 c.PostBody, c.Title, VectorDistance(c.bodyvector,@embedding) AS score FROM c WHERE c.PostType ='Question' ORDER BY VectorDistance(c.bodyvector,@embedding)"
      ).WithParameter("@embedding", embedding);
  FeedIterator<Post> feed = vectorContainer.GetItemQueryIterator<Post>(
      queryDefinition: queryDef
  );
  string results = "";  
  while (feed.HasMoreResults) 
  {
      FeedResponse<Post> response = await feed.ReadNextAsync();      
      foreach (Post item in response)
      {
        results = string.Concat(results, item.PostBody,"\n");
        Console.WriteLine($"Found item:\t{item.score}\t{item.Title}");
      }      
  }
//Console.WriteLine(results);

Found item:	0.7777388909341105	Mechanisms for tracking DB schema changes
Found item:	0.7677463969900357	How to export data from SQL Server 2005 to MySQL
Found item:	0.7651077393713712	How big can a MySQL database get before performance starts to degrade
Found item:	0.763617532519128	Flat file databases
Found item:	0.761934513560742	Auto Generate Database Diagram MySQL
Found item:	0.7583703312360776	Create a SQLite database based on an XSD Data Set
Found item:	0.7567938514574106	Upgrading SQL Server 6.5
Found item:	0.7556035359978679	Using multiple SQLite databases at once
Found item:	0.7549846896313734	Speed Comparisons - Procedural vs. OO in interpreted languages
Found item:	0.754364197857698	Binary Data in MySQL


In [9]:
#r "nuget: Microsoft.SemanticKernel, 1.48.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.48.0"

using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;

Installed Packages Microsoft.SemanticKernel, 1.48.0 Microsoft.SemanticKernel.Connectors.AzureOpenAI, 1.48.0

In [10]:
var builder = Kernel.CreateBuilder();
builder.AddAzureOpenAIChatCompletion("test4o", "https://savranweb.openai.azure.com/", "3Qg3P8Fw37qaN3ZtG2JJapZZVjBFTKvOIDaIYXoL5a5fduu3wz2XJQQJ99BDACYeBjFXJ3w3AAABACOGPUlQ");
Kernel kernel = builder.Build();
var chatcompservice = kernel.Services.GetRequiredService<IChatCompletionService>();

var chathistory = new ChatHistory();
var executionsettings = new OpenAIPromptExecutionSettings { FunctionChoiceBehavior = FunctionChoiceBehavior.Auto() };
executionsettings.ChatSystemPrompt = "You are a helpful assistant. The given text is the result of a vector search. Summarize the text.";


chathistory.AddUserMessage(results);
var result = await chatcompservice.GetChatMessageContentAsync(chathistory, executionsettings, kernel);
result.Content.Display();

The text discusses several database-related challenges and inquiries, including automating DB schema changes, migrating data between SQL Server and MySQL, performance considerations for large MySQL databases, and creating database diagrams. The author seeks solutions for automating database updates while using Subversion, migrating from SQL Server 2005 to MySQL, and improving MySQL performance with large data volumes. They also explore creating SQLite databases from XSD DataSets, upgrading SQL Server 6.5 to 2005, syncing SQLite databases, and choosing between Procedural and Object-Oriented programming in PHP and JavaScript. Additionally, the text touches on storing binary data in MySQL and finding tools for automatically generating database diagrams.